In [30]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdDetermineBonds
from rdkit.Chem.Draw import IPythonConsole
from openff.toolkit import Molecule, Topology, ForceField
from openff.interchange import Interchange
from openff.units import unit
from openff.units.openmm import from_openmm, to_openmm
import openmm
import numpy as np
import sys
from pdb_wizard import PBC
import random
from copy import deepcopy
import scipy

In [3]:
mol = Chem.MolFromXYZFile('./HKUST-1.xyz')

rdDetermineBonds.DetermineConnectivity(mol)
# IPythonConsole.drawMol3D(mol)

In [5]:
editable = Chem.EditableMol(mol)
# No kill like over kill
METALS = [3, 4, 11, 12, 13, 19, 20, 21, 22, 23, 24, 25 ,26, 27, 28, 29, 30, 31, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]

for bond in mol.GetBonds():
    a1 = bond.GetBeginAtom()
    a1_idx = bond.GetBeginAtomIdx()
    a2 = bond.GetEndAtom()
    a2_idx = bond.GetEndAtomIdx()
    if a1.GetAtomicNum() in METALS or a2.GetAtomicNum() in METALS:
        editable.RemoveBond(a1_idx, a2_idx)
    
# IPythonConsole.drawMol3D(editable.GetMol())

In [6]:
# You could change sanitizeFrags to True, but I doubt it's necessary
frags = Chem.GetMolFrags(editable.GetMol(), asMols=True, sanitizeFrags=True)
for frag in frags:
    if frag.GetNumAtoms() == 1:
        charge = 2
    else:
        charge = -3
    rdDetermineBonds.DetermineBonds(frag, charge=charge)

In [7]:
mols = [Molecule.from_rdkit(frag) for frag in frags]

In [8]:
for mol in mols:
    if mol.to_smiles() == '[Cu]':
        mol.atom(0).formal_charge = 2

In [9]:
def append_system(existing_system, system_to_append, cutoff, index_map=None):
    """Appends a system object onto the end of an existing system.

    Parameters
    ----------
    existing_system: openmm.System, optional
        The base system to extend.
    system_to_append: openmm.System
        The system to append.
    cutoff: openff.evaluator.unit.Quantity
        The nonbonded cutoff
    index_map: dict of int and int, optional
        A map to apply to the indices of atoms in the `system_to_append`.
        This is predominantly to be used when the ordering of the atoms
        in the `system_to_append` does not match the ordering in the full
        topology.
    """
    supported_force_types = [
        openmm.HarmonicBondForce,
        openmm.HarmonicAngleForce,
        openmm.PeriodicTorsionForce,
        openmm.NonbondedForce,
        openmm.RBTorsionForce,
        openmm.CustomNonbondedForce,
        openmm.CustomBondForce,
    ]

    number_of_appended_forces = 0
    index_offset = existing_system.getNumParticles()

    # Create an index map if one is not provided.
    if index_map is None:
        index_map = {i: i for i in range(system_to_append.getNumParticles())}

    # Append the particles.
    for index in range(system_to_append.getNumParticles()):
        index = index_map[index]
        existing_system.addParticle(system_to_append.getParticleMass(index))

    # Append the constraints
    for index in range(system_to_append.getNumConstraints()):
        index_a, index_b, distance = system_to_append.getConstraintParameters(index)

        index_a = index_map[index_a]
        index_b = index_map[index_b]

        existing_system.addConstraint(
            index_a + index_offset, index_b + index_offset, distance
        )

    # Validate the forces to append.
    for force_to_append in system_to_append.getForces():
        if type(force_to_append) in supported_force_types:
            continue

        raise ValueError(
            f"The system contains an unsupported type of "
            f"force: {type(force_to_append)}."
        )

    # Append the forces.
    for force_to_append in system_to_append.getForces():
        existing_force = None

        for force in existing_system.getForces():
            if type(force) not in supported_force_types:
                raise ValueError(
                    f"The existing system contains an unsupported type "
                    f"of force: {type(force)}."
                )

            if type(force_to_append) is not type(force):
                continue

            if isinstance(
                force_to_append, openmm.CustomNonbondedForce
            ) or isinstance(force_to_append, openmm.CustomBondForce):
                if force_to_append.getEnergyFunction() != force.getEnergyFunction():
                    continue

            existing_force = force
            break

        if existing_force is None:
            if isinstance(force_to_append, openmm.CustomNonbondedForce):
                existing_force = openmm.CustomNonbondedForce(
                    force_to_append.getEnergyFunction()
                )
                existing_force.setCutoffDistance(cutoff)
                existing_force.setNonbondedMethod(
                    openmm.CustomNonbondedForce.CutoffPeriodic
                )
                for index in range(force_to_append.getNumGlobalParameters()):
                    existing_force.addGlobalParameter(
                        force_to_append.getGlobalParameterName(index),
                        force_to_append.getGlobalParameterDefaultValue(index),
                    )
                for index in range(force_to_append.getNumPerParticleParameters()):
                    existing_force.addPerParticleParameter(
                        force_to_append.getPerParticleParameterName(index)
                    )
                existing_system.addForce(existing_force)

            elif isinstance(force_to_append, openmm.CustomBondForce):
                existing_force = openmm.CustomBondForce(
                    force_to_append.getEnergyFunction()
                )
                for index in range(force_to_append.getNumGlobalParameters()):
                    existing_force.addGlobalParameter(
                        force_to_append.getGlobalParameterName(index),
                        force_to_append.getGlobalParameterDefaultValue(index),
                    )
                for index in range(force_to_append.getNumPerBondParameters()):
                    existing_force.addPerBondParameter(
                        force_to_append.getPerBondParameterName(index)
                    )
                existing_system.addForce(existing_force)

            else:
                existing_force = type(force_to_append)()
                existing_system.addForce(existing_force)

        if isinstance(force_to_append, openmm.HarmonicBondForce):
            # Add the bonds.
            for index in range(force_to_append.getNumBonds()):
                index_a, index_b, *parameters = force_to_append.getBondParameters(
                    index
                )

                index_a = index_map[index_a]
                index_b = index_map[index_b]

                existing_force.addBond(
                    index_a + index_offset, index_b + index_offset, *parameters
                )

        elif isinstance(force_to_append, openmm.HarmonicAngleForce):
            # Add the angles.
            for index in range(force_to_append.getNumAngles()):
                (
                    index_a,
                    index_b,
                    index_c,
                    *parameters,
                ) = force_to_append.getAngleParameters(index)

                index_a = index_map[index_a]
                index_b = index_map[index_b]
                index_c = index_map[index_c]

                existing_force.addAngle(
                    index_a + index_offset,
                    index_b + index_offset,
                    index_c + index_offset,
                    *parameters,
                )

        elif isinstance(force_to_append, openmm.PeriodicTorsionForce):
            # Add the torsions.
            for index in range(force_to_append.getNumTorsions()):
                (
                    index_a,
                    index_b,
                    index_c,
                    index_d,
                    *parameters,
                ) = force_to_append.getTorsionParameters(index)

                index_a = index_map[index_a]
                index_b = index_map[index_b]
                index_c = index_map[index_c]
                index_d = index_map[index_d]

                existing_force.addTorsion(
                    index_a + index_offset,
                    index_b + index_offset,
                    index_c + index_offset,
                    index_d + index_offset,
                    *parameters,
                )

        elif isinstance(force_to_append, openmm.NonbondedForce):
            # Add the vdW parameters
            for index in range(force_to_append.getNumParticles()):
                index = index_map[index]

                existing_force.addParticle(
                    *force_to_append.getParticleParameters(index)
                )

            # Add the 1-2, 1-3 and 1-4 exceptions.
            for index in range(force_to_append.getNumExceptions()):
                (
                    index_a,
                    index_b,
                    *parameters,
                ) = force_to_append.getExceptionParameters(index)

                index_a = index_map[index_a]
                index_b = index_map[index_b]

                existing_force.addException(
                    index_a + index_offset, index_b + index_offset, *parameters
                )

        elif isinstance(force_to_append, openmm.RBTorsionForce):
            # Support for RBTorisionForce needed for OPLSAA, etc
            for index in range(force_to_append.getNumTorsions()):
                torsion_params = force_to_append.getTorsionParameters(index)
                for i in range(4):
                    torsion_params[i] = index_map[torsion_params[i]] + index_offset

                existing_force.addTorsion(*torsion_params)

        elif isinstance(force_to_append, openmm.CustomNonbondedForce):
            for index in range(force_to_append.getNumParticles()):
                nb_params = force_to_append.getParticleParameters(index_map[index])
                existing_force.addParticle(nb_params)

            # Add the 1-2, 1-3 and 1-4 exceptions.
            for index in range(force_to_append.getNumExclusions()):
                (
                    index_a,
                    index_b,
                ) = force_to_append.getExclusionParticles(index)

                index_a = index_map[index_a]
                index_b = index_map[index_b]

                existing_force.addExclusion(
                    index_a + index_offset, index_b + index_offset
                )

        elif isinstance(force_to_append, openmm.CustomBondForce):
            for index in range(force_to_append.getNumBonds()):
                index_a, index_b, bond_params = force_to_append.getBondParameters(
                    index
                )

                index_a = index_map[index_a] + index_offset
                index_b = index_map[index_b] + index_offset

                existing_force.addBond(index_a, index_b, bond_params)

        number_of_appended_forces += 1

    if number_of_appended_forces != system_to_append.getNumForces():
        raise ValueError("Not all forces were appended.")

def create_empty_system():
    system = openmm.System()
    system.addForce(openmm.HarmonicBondForce())
    system.addForce(openmm.HarmonicAngleForce())
    system.addForce(openmm.PeriodicTorsionForce())

    nonbonded_force = openmm.NonbondedForce()
    nonbonded_force.setNonbondedMethod(openmm.NonbondedForce.NoCutoff)

    system.addForce(nonbonded_force)

    return system

In [23]:
TEMP = to_openmm(298 * unit.kelvin)
PRESSURE = to_openmm(1 * unit.atm)
TIMESTEP = to_openmm(2 * unit.femtosecond)
CO2 = Molecule.from_smiles('O=C=O')
BOLTZMANN = openmm.unit.BOLTZMANN_CONSTANT_kB
OPENFF_FF = ForceField("openff-2.2.0-uff.offxml", load_plugins=True)
PROB_INSERT_DELETE = 0.333
R_CUTOFF = 0.4 * unit.nanometer


In [31]:
MOF_TOP = Topology.from_molecules(mols)   
BOX_SIZE = 2.6343  # nm 
MOF_TOP.box_vectors = np.array([[BOX_SIZE, 0, 0], [0, BOX_SIZE, 0], [0, 0, BOX_SIZE]]) * unit.nanometer
MOF_TOP.is_periodic = True

interchange = Interchange.from_smirnoff(topology=MOF_TOP, force_field=OPENFF_FF)
MOF_OPENMM_SYS = interchange.to_openmm(combine_nonbonded_forces=False)
for i in range(MOF_OPENMM_SYS.getNumParticles()):
    MOF_OPENMM_SYS.setParticleMass(i, 0.0)

In [24]:
def monte_carlo_test(E_new, E_old):
    delta_E = E_new - E_old
    probability = np.exp(-delta_E / (BOLTZMANN * TEMP))
    random_number = random.uniform(0, 1)
    
    return probability >= random_number

def system_energy(gases: np.ndarray) -> float:
    gas_mols = [Molecule.from_smiles(CO2) for _ in gases]
    gas_top = Topology.from_molecules(gas_mols)
    gas_top.is_periodic = True
    gas_top.is_periodic = True
    gas_interchange = Interchange.from_smirnoff(topology=gas_top, force_field=OPENFF_FF)
    gas_openmm_sys = gas_interchange.to_openmm(combine_nonbonded_forces=False)

    mof_openmm_sys = deepcopy(MOF_OPENMM_SYS)
    append_system(mof_openmm_sys, gas_openmm_sys)

    # openmm_sys.addForce(openmm.MonteCarloBarostat(PRESSURE, TEMP))
    openmm_integrator = openmm.LangevinIntegrator(TEMP, 1, TIMESTEP)

    openmm_sim = openmm.app.Simulation(MOF_TOP.to_openmm(), mof_openmm_sys, openmm_integrator, platform=openmm.Platform.getPlatformByName("CPU"))
    reporter = openmm.app.StateDataReporter(sys.stdout, 1000, step=True, time=True, potentialEnergy=True,
                                            kineticEnergy=True, totalEnergy=True, temperature=True, speed=True, density=True)
    openmm_sim.reporters.append(reporter)
    openmm_sim.reporters.append(openmm.app.PDBReporter('out.pdb', 1))
    openmm_sim.context.setPositions(to_openmm(MOF_TOP.get_positions()))
    # openmm_sim.context.setVelocitiesToTemperature(TEMP)
    openmm_sim.minimizeEnergy()
    context = openmm_sim.context.getState(getEnergy=True)
    return context.getPoteintialEnergy().value_in_unit(openmm.unit.kilojoules_per_mole)

def insert_gas(gas_positions):
    CO2_POS = [
        [0, 0, 0] * unit.nanometer,
        [0.1163, 0, 0] * unit.nanometer,
        [0.2326, 0, 0] * unit.nanometer
    ]
    while True:
        rotation_matrix = scipy.spatial.transform.Rotation.random().as_matrix()
        shift = np.array([random.uniform(0, BOX_SIZE), random.uniform(0, BOX_SIZE), random.uniform(0, BOX_SIZE)]) * unit.nanometer
        new_pos = np.dot(gas_positions, rotation_matrix) + shift
        if np.all(np.linalg.norm(new_pos, axis=1) < BOX_SIZE):
            break


In [ ]:
random.seed(42)
gases = []  # Must be a list where elements are [x, y, z] * unit.nanometer coordinates
insert_gas(gases)
for timestep in range(10):
    operation = random.random()
    if operation < PROB_INSERT_DELETE:  # Insert
        pass
    elif operation < 2 * PROB_INSERT_DELETE:  # Delete
        pass
    else:  # Translate
        pass


[1.4530999999999998 1.796 0.6418999999999999] nanometer
[1.796 1.4530999999999998 0.6418999999999999] nanometer
[1.796 1.9568999999999996 0.1381] nanometer
[1.4530999999999998 1.9568999999999996 0.4809999999999999] nanometer
[1.9568999999999996 1.4530999999999998 0.4809999999999999] nanometer
[1.9568999999999996 1.796 0.1381] nanometer
[1.6703999999999999 1.6703999999999999 0.5265] nanometer
[1.6703999999999999 1.8414999999999997 0.35539999999999994] nanometer
[1.8414999999999997 1.6703999999999999 0.35539999999999994] nanometer
[1.6271999999999998 1.6271999999999998 0.6172] nanometer
[1.6271999999999998 1.9321 0.3122] nanometer
[1.9321 1.6271999999999998 0.3122] nanometer
[1.4970999999999999 1.8501999999999998 0.5352] nanometer
[1.8501999999999998 1.4970999999999999 0.5352] nanometer
[1.8501999999999998 1.8501999999999998 0.18209999999999998] nanometer
[1.6135999999999997 1.7844 0.46939999999999993] nanometer
[1.7844 1.6135999999999997 0.46939999999999993] nanometer
[1.7844 1.7844 0.2